Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

In [1]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-12 04:24:57--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-12 04:24:57--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc814bb54519a9b13050cac17af4.dl.dropboxusercontent.com/cd/0/inline/BnBXBwxCXv2zZpte32386v4O6BONswO6TorcvXvf7M5e3QVG84JkCkZ5-mixcRZjlM-KehuTfdHt9d3MW8aubgP4-leWB0h89WFNocYEUavdns8RL5-HcTjfNlQZIXQWVpkCqrHSNRm1TQ43MX6-R3m4aWLbRmKviBqAimqotrXXkw/file# [following]
--2022-06-12 04:24:58--  https://uc814bb54519a9b13050cac17af4.dl.dropboxusercontent.com/cd/0/inline/BnBXBwxCXv2zZpte32386v4O6BONswO6TorcvXvf7M5e3QVG84JkCkZ5-mixcRZjlM-K

## Что надо сделать

1. объединить в одну выборку 

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
data = positive.append(negative)

In [4]:
data.tail()

,text
111918,Но не каждый хочет что то исправлять:( http://...
111919,скучаю так :-( только @taaannyaaa вправляет мо...
111920,"Вот и в школу, в говно это идти уже надо("
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :..."
111922,Такси везет меня на работу. Раздумываю приплат...


2. провести исследование и выявить тематики о которых говорят в твитах 

In [5]:
import re
import numpy as np
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool
from pymystem3 import Mystem

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 8.2 MB 10.7 MB/s 


In [8]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Пишем функции по предобработке текста

In [16]:
import nltk
nltk.download('averaged_perceptron_tagger_ru')

words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')

#Дополним список стоп-слов
add_stop_words = ['ахах', 'мочь', 'супер', 'хотяб', 'ахахаххах', 'аахха', 'настюха',
                  'оля', 'блин']
stopwords_list += add_stop_words

# mystem = Mystem()
# def lemmatize(words, lemmer = mystem, stopwords = stopwords_list):
#     lemmas = lemmer.lemmatize(' '.join(words))
#     return [w for w in lemmas if not w in stopwords 
#             and w.isalpha()]

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

# Добавим также выделение частей речи в токенах, чтобы в дальнейшем отфильтровать
def preprocess(text):
    result = (lemmatize(find_words(text)))
    return nltk.pos_tag(result, lang='rus')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


Пример предобработки

In [17]:
data.text.iloc[1]

'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D'

In [18]:
print(preprocess(data.text.iloc[1]))

[('всё', 'S-PRO'), ('таки', 'PART'), ('немного', 'ADV'), ('похожий', 'A=m'), ('мальчик', 'S'), ('всё', 'S-PRO'), ('равно', 'A=n'), ('хороший', 'A=m')]


Предобработаем вес корпус твитов

In [55]:
#data = data.sample(40000)

In [19]:
preprocessed_text = list(tqdm(map(preprocess, data.text), total=len(data)))

  0%|          | 0/226834 [00:00<?, ?it/s]

In [20]:
preprocessed_text[:5]

[[('школотый', 'A=m'),
  ('поверь', 'S'),
  ('самый', 'A-PRO=m'),
  ('общество', 'S'),
  ('профилировать', 'V'),
  ('предмет', 'S'),
  ('тип', 'S')],
 [('всё', 'S-PRO'),
  ('таки', 'PART'),
  ('немного', 'ADV'),
  ('похожий', 'A=m'),
  ('мальчик', 'S'),
  ('всё', 'S-PRO'),
  ('равно', 'A=n'),
  ('хороший', 'A=m')],
 [('katiacheh', 'NONLEX'), ('идиотка', 'S'), ('испугаться', 'V')],
 [('угол', 'S'),
  ('сидеть', 'V'),
  ('погибать', 'V'),
  ('голод', 'S'),
  ('ещё', 'ADV'),
  ('порция', 'S'),
  ('взять', 'V'),
  ('хотя', 'PART'),
  ('жрать', 'V'),
  ('хотеть', 'V'),
  ('http', 'NONLEX')],
 [('значит', 'V'),
  ('страшилка', 'S'),
  ('посмотреть', 'V'),
  ('всё', 'ADV'),
  ('часть', 'S'),
  ('создаться', 'V'),
  ('ощущение', 'S'),
  ('автор', 'S'),
  ('курить', 'V')]]

Создадим функцию, которая будет оставлять только существительные из всего корпуса

In [21]:
def only_nouns(preprocessed_text):
    token_list = []
    for tweet in preprocessed_text:
        token_list.append([w[0] for w in tweet if w[1]=='S'])
    return token_list


Воспользуемся этой функцией для нашего корпуса

In [22]:
data['text'] = only_nouns(preprocessed_text=preprocessed_text)
data.sample(8)


,text
8597,[]
15061,"[уха, клёв, ахи]"
6382,"[час, момент]"
36619,[]
103932,[мнение]
81289,"[шанс, дверь, главное, ключик, надежда, олег, ..."
71813,"[андрей, бедняк, пост, программа, орёл, решка]"
98256,"[контакт, катастрофа]"


Отфильтруем твиты, где оказалось меньше 3 существительных

In [23]:
data = data[data['text'].map(lambda d: len(d)) > 2]

In [24]:
data.head(5)

,text
0,"[поверь, общество, предмет, тип]"
3,"[угол, голод, порция]"
4,"[страшилка, часть, ощущение, автор]"
14,"[обзор, версия, мусор]"
15,"[аспирантура, наука, добро]"


Итого осталось следующее количество твитов для обучения

In [25]:
len(data)

96640

## Создаем модель

In [26]:
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 63.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [27]:
from gensim.models import *
from gensim import corpora

Модель использует векторное представление документов, например, мешок слов (bow), поэтому сперва создадим словарь:

In [28]:
dictionary = corpora.Dictionary(data['text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('tweet.dict')

Векторизуем твиты:

In [29]:
corpus = [dictionary.doc2bow(text) for text in data['text']]
corpora.MmCorpus.serialize('tweet.model', corpus) 

Обучаем модель

In [43]:
%time lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3, chunksize=200, update_every=1, passes=2)

CPU times: user 26.6 s, sys: 515 ms, total: 27.1 s
Wall time: 27 s


Смотрим топ-слова в получивщихся тематиках и их распределение

In [44]:
lda.show_topics(num_topics=5, num_words=10, formatted=True)

[(0,
  '0.020*"ночь" + 0.017*"неделя" + 0.013*"снег" + 0.012*"фотка" + 0.012*"фильм" + 0.012*"боль" + 0.012*"вечер" + 0.011*"бог" + 0.010*"пиздец" + 0.010*"место"'),
 (1,
  '0.062*"день" + 0.030*"человек" + 0.024*"дом" + 0.024*"час" + 0.020*"время" + 0.019*"утро" + 0.015*"работа" + 0.014*"урок" + 0.013*"бля" + 0.012*"улица"'),
 (2,
  '0.033*"год" + 0.027*"школа" + 0.026*"жизнь" + 0.020*"мама" + 0.015*"голова" + 0.013*"экзамен" + 0.011*"рука" + 0.010*"учёба" + 0.009*"конец" + 0.009*"чувство"')]

3. сделать визуализацию кластеров тематик

In [35]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=8c869495876de142111e582f005c4c8f286404a304f4b0307ed912619d3bc547
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [45]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%time vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


CPU times: user 6.83 s, sys: 104 ms, total: 6.93 s
Wall time: 7.12 s


4. проинтерпритировать получившиеся тематики

## Достаточно сложная задача )

#### Но добавив немного вооображения можно представить так:

*тема 1*: твиты посвященные ежедневной рутине (дела, работа, учеба, уход за детьми и т.п.)

*тема 2*: твиты, посвященные каким-то жизненным событиям, которые вызывают сильные эмоции

*тема 3*: твиты, посвященные описанию того, как проводится досуг (фильмы, фото, знакомства, книги и т.п.)